# Transition-Based Constrained DFT in BigDFT

This notebook showcases a full workflow for computing vertical excitation energies for the low lying states of molecules using the transition-based CDFT (T-CDFT) approach within the linear scaling formalism in BigDFT. This follows the methodology described in M. Stella, K. Thapa, L. Genovese and L. E. Ratcliff, J. Chem Theory Comput. 18, 3027 (2022).

We present an example workflow for naphthalene. For the purposes of reducing the computational cost, we use a minimal support function (SF) basis. Note that for accurate convergence of the results, a larger basis would be required.

The first step is the generation of the fixed SF basis which will be later used for running T-CDFT calculations. The workflow also computes a simple descriptor for characterising the nature of the selected excited state (charge-transfer or local excitation) and a TDDFT analysis to identify whether the excitation is e.g. purely HOMO-LUMO or if it involves multiple orbitals, e.g HOMO to LUMO and HOMO to LUMO+1.

This tutorial assumes that you are familiar with the basics of PyBigDFT and BigDFT, including the system class, linear scaling BigDFT, and running calculations. The T-CDFT calculation also uses the same file structure as fragment calculations, and thus it may also be useful to look at the fragment tutorial.

Notebook by Dr. Martina Stella and Dr. Laura Ratcliff. 

## STEP 1: Generate Support Function Basis

Our first step is to generate SFs which are suitable for excited state calculations. This involves two calculations, the first of which will use the standard linear scaling protocol. The SFs and density from this calculation are then used as an input for a non-self-consistent calculation, which will also optimise the SFs to represent a given number of virtual states.

First we make the BigDFT input files, where we will have a separate input dictionary for each of the two calculations.

In [1]:
from BigDFT import Inputfiles as I, InputActions as A
import sf_sizes

inpl = I.Inputfile()
inpv = I.Inputfile()

# set the XC functional and basis set parameters
xc = 'PBE'
hgrid = 0.5
crmult = 5
frmult = 7
rloc = 8
basis_size = 's_sp'

# set the common parameters
for inp in [inpl, inpv]:
    inp.set_hgrid(hgrid)
    inp.set_rmult(coarse=crmult, fine=frmult) 
    inp.set_xc(xc)
    inp.set_psp_nlcc()
    inp['import'] = 'linear'
    inp.update({'perf': {'check_sumrho': 0, 'check_overlap': 0}})
    inp.update({'lin_general': {'rpnrm_cv': 1.0e-11, 'output_wf': 1, 'output_mat': 1,
                                'charge_multipoles': 0}})
    # set up the basis, making sure the kernel cutoff is much larger than the SF radius
    lin_basis_dict = sf_sizes.set_support_function_dict(rloc, rloc + 10, basis_size)
    inp.update(lin_basis_dict)
    
# linear calculation for ground state
inpl.update({'lin_kernel': {'linear_method': 'DIAG', 'alphamix': 0.1}})
    
# non-self-consistent linear calculations for virtual state(s)
inpv['dft'].update({'inputpsiid': 'linear_restart'}) 
inpv['perf'].update({'adjust_kernel_iterations': False, 'adjust_kernel_threshold': False})
inpv['lin_general'].update({'hybrid': False, 'kernel_restart_mode': 'coeff', 'nit': [0, 200],
                            'rpnrm_cv': 1.0e-8})
inpv.update({'lin_basis': {'nit': 4, 'idsx': 2, 'fix_basis': 0.0, 'gnrm_cv': 1.0e-2}})
inpv.update({'lin_kernel': {'nit': 1, 'rpnrm_cv': 1.0e-8, 'delta_pnrm': -1, 'linear_method': 'DIRMIN', 
                            'alphamix': 0.0, 'nstep':  500, 'gnrm_cv_coeff': 1.0e-4}})
inpv.update({'output': {'orbitals': 'text'}})

# set the number of virtual states we want to optimise
# this should be big enough to include all virtual states which are involved in the excitation
# but should not include high energy delocalised virtual states, 
# since these are challenging to represent in a localised basis
# here, we choose the number so as to correspond to the number of negative states in a cubic scaling calculation
nvirt = 3
inpv['lin_general'].update({'extra_states': nvirt})
inpv.update({'mix': {'norbsempty': nvirt}})

Next we set up the BigDFT calculator.

In [2]:
from BigDFT import Calculators as C
code = C.SystemCalculator(omp=2, mpi_run='mpirun -n 16', verbose=False, skip=True)

Now we have everything we need to run the calculations.

In [3]:
from BigDFT import Logfiles as lf
import sf_sizes

Ha2eV = 27.211396132

# the name of the xyz file
molecule = 'naphthalene'

# both calculations will use the same data directory
# so let's name it using the key calculation parameters
radical = molecule+'_'+str(xc)+'_S0_SF_'+basis_size+'_'+str(rloc)

basis_types = ['linopt', 'virtopt']

for lv,linvirt in enumerate(basis_types):
    basis = linvirt+'_'+basis_size+'_'+str(rloc)
    name = molecule+'_'+str(xc)+'_S0_'+basis

    # run the calculation, setting the appropriate radical
    if linvirt == 'linopt':
        inpl.update({'radical': radical})
        lin_run = code.run(input=inpl, posinp=molecule+'.xyz', name=name)
        run = lin_run
    else:
        inpv.update({'radical': radical})   
        virt_run = code.run(input=inpv, posinp=molecule+'.xyz', name=name) 
        run = virt_run
           
    # let's also summarise some of the results
    energy = Ha2eV * run.energy / run.nat
                
    if linvirt == 'linopt':
        ihomo = run.log['Total Number of Orbitals']
                    
    homo = Ha2eV * run.evals[0][0][ihomo - 1]
    lumo = Ha2eV * run.evals[0][0][ihomo]
    time = run.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
    print(molecule+' with '+basis+' took '+'{0:.1f}'.format(time)+' minutes, E = '+\
          '{0:.2f}'.format(energy)+' eV/atom, HOMO = '+'{0:.2f}'.format(homo)+' eV, LUMO = '+\
          '{0:.2f}'.format(lumo)+' eV')

naphthalene with linopt_s_sp_8 took 2.0 minutes, E = -106.58 eV/atom, HOMO = -5.45 eV, LUMO = -1.22 eV
naphthalene with virtopt_s_sp_8 took 0.6 minutes, E = -106.57 eV/atom, HOMO = -5.44 eV, LUMO = -2.04 eV


We can see from the above results that the HOMO does not change much between the ground state and virtual calculations, however the LUMO decreases in energy significantly. This is because the LUMO was not well represented in the ground state calculation.

The total energy increases slightly, as it is more challenging to also represent virtual states in the same basis - the larger the basis, the smaller this increase would be.

## STEP 2: Calculate Transition Breakdown using TDDFT

Now that we have our basis, we want to determine the transition breakdown, i.e. which transitions contribute to the targeted excitations. To do this, we will use TDDFT.

Although TDDFT is performed using cubic scaling BigDFT, we can use the KS wavefunction files which we already generated for a cubic scaling calculation. In other words, we can skip the wavefunction optimisation step of BigDFT, and jump straight to the TDDFT calculation.

However, the virtual wavefunctions need renaming before we can use them for TDDFT, while the orbital index also needs editing. Let's make a function to do this manipulation.

In [4]:
def wavefunctions_to_virtuals(radical, ihomo, nvirt=1):
    import os
    import shutil
    import subprocess
        
    os.chdir('data-'+radical)
                        
    # loop over unoccupied states
    for vsf in range(nvirt):
        vsfstr = str(vsf + 1).zfill(6)
        isf = ihomo + vsf
        isfstr = str(isf + 1).zfill(6)
        if not os.path.exists('virtuals-k001-NR.b'+vsfstr):
            shutil.copy('wavefunction-k001-NR.b'+isfstr,
                        'virtuals-k001-NR.b'+vsfstr)
            
        # finally, we need to modify the first number in virtual state files,
        # since the first virtual orbital needs to be 1
        # but first check whether or not we did this already
        COMMAND = "awk 'NR==1{print $1}' $1"
        ind = int(subprocess.check_output([COMMAND, "sh", 'virtuals-k001-NR.b'+vsfstr], shell=True))
        if ind == isf + 1:
            COMMAND = 'i='+str(isf+1)+'; j='+str(vsf+1)+'; sed -i -e "1s/$i /$j /" $1'
            out = subprocess.check_output([COMMAND, "sh", 'virtuals-k001-NR.b'+vsfstr], shell=True)
            
    os.chdir('../')

    
# the number of virtual states should equal the basis size minus the number of occupied states
sf_dict = sf_sizes.set_support_function_dict(-1, -1, basis_size)
nbasis = sf_sizes.get_number_of_sfs(lin_run, sf_dict)
nocc = lin_run.number_of_orbitals
nvirt_tddft = nbasis - nocc
wavefunctions_to_virtuals(radical, ihomo, nvirt=nvirt_tddft)

Now that we have the wavefunctions in the correct format, let's define our input dictionary. We want the same wavelet parameters as for the linear scaling calculations.

In [5]:
inpt = I.Inputfile()
inpt.set_hgrid(hgrid)
inpt.set_rmult(coarse=crmult, fine=frmult) 
inpt.set_psp_nlcc()
# TDDFT can currently only be performed using the ABINIT version of LDA (not LibXC)
inpt.set_xc(1)  
inpt['dft'].update({'inputpsiid': 2, 'itermax': 1}) 
inpt.update({'tddft': {'tddft_approach':'full'}})

# use the existing data directory
radical = molecule+'_'+xc+'_S0_SF_'+basis_size+'_'+str(rloc)  
inpt.update({'radical': radical})
inpt.extract_virtual_states(nvirt_tddft, davidson=True, norbv=nvirt_tddft, itermax_virt=1)

Before we run the calculation, let's also define a function which will help with post-processing the results.

In [6]:
def decompose_excitation(np, nalpha, evec, tol):
    """
    Express the excitaion in the basis of the KS transitions, according to a given tolerance
    
    Args:
        np (tuple): (norbu, norbd) occupied orbitals: when of length 1 assumed spin averaged
        nalpha (tuple): (norbu, norbd) virtual orbitals: when of length 1 assumed spin averaged
        evec (array): the components of the excitation vector
        tol (float): the tolerance to be used to determine whather a transition contributes to the excitation
    """

    transitions = []
    considered_norm = 0.0
    for ispin in [0, 1]:
        jspin = ispin if len(nalpha) == 2 else 0
        
        for iorb in range(np[jspin]):
            
            for ialpha in range(nalpha[jspin]):
                index = LR.transition_indexes(np, nalpha, [[iorb, ialpha, ispin]])
                val = evec[0, index[0]]
                
                if val**2 > tol: 
                    transitions.append({'ovsigma': [-np[jspin]+iorb+1, ialpha, ispin], 'W': val})
                    considered_norm += val**2
                    
    return transitions, considered_norm, 1.0 - considered_norm < tol

Now we're ready to run the TDDFT calculation using the pre-existing basis.

In [7]:
from BigDFT import LRTDDFT as LR
import numpy as np

# the number of excitations we are interested, for now the lowest 3
num_excitations = 3

# whether or not to produce some of the more detailed information
plot_doe = False
print_decomposition = False

name = molecule+'_LDA_S0_SF_'+basis_size+'_'+str(rloc)+'_tddft'
tddft_run = code.run(input=inpt, posinp=molecule+'.xyz', name=name)   
time = tddft_run.log['Timings for root process']['Elapsed time (s)'] / 60.0  

print('TDDFT calculation for '+molecule+' with LDA and '+basis_size+\
      ' and Rloc = '+str(rloc)+' took '+'{0:.1f}'.format(time)+' minutes')

# load coupling matrix and transition multipoles from the TDDFT outputs
cm = LR.CouplingMatrix(log=tddft_run)
tm = LR.TransitionMultipoles(log=tddft_run)

# extract excitation energies
evals = tddft_run.evals
ex = LR.Excitations(cm=cm, tm=tm)
ex.split_excitations(evals=evals, tol=1.e-2)
ex.identify_singlet_and_triplets(tol=1.e-2)
eigenvalues = ex.eigenvalues 

st = ['singlets', 'triplets']

# extract data to plot density of excitations
if plot_doe:
    DoE = ex.dos(label='full')
    for state in st:
        tmp = ex.dos_dict(group=state)
        tmp.update(label=state)
        DoE.append(**tmp)
    DoE.plot(xlmax=10, ylmax=500, ylmin=0, sigma=0.1)

# extract energies and decomposition for both singlets and triplets
excitations = {}
for state in st:
    excitations[state] = []
    lu = ex.lookup(group=state)
    if print_decomposition:
        print(state+': ')
    for i in range(num_excitations):
        energy = np.sqrt(ex.eigenvalues[lu][i]) * Ha2eV
        if print_decomposition:
            print('{0:.2f}'.format(energy), ' ', end=' ') 
        decomposition = decompose_excitation(cm.norb_occ, cm.norb_virt, ex.eigenvectors[lu][i], 5.e-4)
        if print_decomposition:
            print(decomposition)
        excitations[state].append({'energy': energy, 'decomposition': decomposition}) 

TDDFT calculation for naphthalene with LDA and s_sp and Rloc = 8 took 7.2 minutes
Loading data with  (24,)  occupied and  (24,)  empty states, from file " /Users/martinastella/Desktop/leeds_tutorials/naphthalene_tcdft/data-naphthalene_PBE_S0_SF_s_sp_8/coupling_matrix.txt "
Using pandas:
done
Shape is conformal with the number of orbitals
Casida Matrix is symmetric True
Loading data with  (24,)  occupied and  (24,)  empty states, from file " /Users/martinastella/Desktop/leeds_tutorials/naphthalene_tcdft/data-naphthalene_PBE_S0_SF_s_sp_8/transition_quantities.txt "
Using pandas:
done
Shape is conformal with the number of orbitals
Diagonalizing Coupling matrix of shape (1152, 1152)
Eigensystem solved


Finally, let's display the results in a table.

In [8]:
import pandas as pd
from IPython.display import display

def get_table(state, num_excitations, excitations, num_orbitals, spin):
    
    occ_orbitals = ['']
    virt_orbitals = ['Energy']
    for iocc in range(num_orbitals):
        for jvirt in range(num_orbitals):
            if iocc == 0:
                occ_orbitals.append('HOMO')
            else:
                occ_orbitals.append('HOMO-'+str(iocc))
            if jvirt == 0:
                virt_orbitals.append('LUMO')
            else:
                virt_orbitals.append('LUMO+'+str(jvirt))
    occ_orbitals.append('Sum')            
    virt_orbitals.append('Sum')          
            
    arrays = [occ_orbitals, virt_orbitals]
    columns = pd.MultiIndex.from_arrays(arrays, names=['', ''])

    if state == 'singlets':
        tag = 'S'
    else:
        tag = 'T'
    rows = [tag + str(i + 1) for i in range(num_excitations)]    

    table_data = []
    
    for i in range(num_excitations):
        table_row = []
        energy = excitations[state][i]['energy']
        table_row.append('{0:.2f}'.format(energy))
        
        # get breakdown of contributions
        decomposition = excitations[state][i]['decomposition'][0]
        
        exc_contrib = np.array([])    
        for i in range(len(decomposition)):
            exc_contrib = np.append(exc_contrib, decomposition[i]['W'])    
        
        sum_contributions = 0.0
        for iocc in range(num_orbitals):
            for jvirt in range(num_orbitals):
                
                pi = ''
                for dec in decomposition:
                    if dec['ovsigma'][0] == -iocc and dec['ovsigma'][1] == jvirt and dec['ovsigma'][2] == spin:
                        # take the square, but also retain the sign
                        pi = dec['W'] * dec['W'] / (np.sum(exc_contrib**2) / 2.0)
                        sum_contributions += pi
                        pi = '{0:.3f}'.format(np.sign(dec['W']) * pi)
                        break
                        
                table_row.append(pi)
                              
        table_row.append('{0:.3f}'.format(sum_contributions))
                
        table_data.append(table_row)
        
    table = pd.DataFrame(table_data, index=rows, columns=columns)

    return table


# summarise the transition breakdown for a given spin channel,
# include transition contributions up to HOMO-/LUMO+(norb - 1) orbitals
spin = 0
norb = 3
singlets_table = get_table('singlets', num_excitations, excitations, norb, spin)
display(singlets_table)

print('')
        
triplets_table = get_table('triplets', num_excitations, excitations, norb, spin)
display(triplets_table)

HOMO                HOMO-1                HOMO-2                 \
   Energy    LUMO LUMO+1  LUMO+2   LUMO LUMO+1 LUMO+2    LUMO LUMO+1  LUMO+2   
S1   4.13  -0.915                        0.028                        -0.050   
S2   4.26          0.507          0.493                                        
S3   5.00                 -0.603                       -0.397                  

      Sum  
      Sum  
S1  0.993  
S2  1.000  
S3  1.000

HOMO                 HOMO-1               HOMO-2                \
   Energy    LUMO  LUMO+1 LUMO+2    LUMO LUMO+1 LUMO+2   LUMO LUMO+1 LUMO+2   
T1   3.06  -0.991                         0.004                       0.004   
T2   3.92          -0.498          0.502                                      
T3   4.06          -0.502         -0.498                                      

      Sum  
      Sum  
T1  0.999  
T2  1.000  
T3  1.000

The above tables shows the energies and transition breakdowns for the 3 lowest singlet and triplet states. We have chosen to only show the contributions coming from the 3 highest occupied and 3 lowest unoccupied molecular orbitals. 

As can be seen, there is an additional small contribution from other orbitals for the lowest singlet state. We can also see that both $S_1$ and $T_1$ are dominated by the HOMO-LUMO transition, with S$_1$ also including more significant contributions from other transitions. This information will be used later on when defining the constraint to impose.

## STEP 3: Calculate HOMO-LUMO Spatial Overlap

Our next step in characterising the excitation is to calculate the spatial overlap between the HOMO and the LUMO. For a pure HOMO-LUMO excitation, this quantity is indicative of the charge transfer nature of the excitation - a high value indicates a strongly local excitation, while a small value indicates a charge transfer excitation.

This step uses the cubetools package, so we need cube files of the HOMO and LUMO. At the end of the virtual calculation, we already wrote the KS wavefunctions to file, so now we need to convert them to cube format. We can do that using BigDFT-tool - let's define a function to wrap the process.

In [9]:
def generate_cube(molecule, log, name, iorb, tag='wavefunction'):
    import os
    import shutil
    
    data_dir = log.data_directory
    filename = data_dir+'/'+tag+'-k001-NR.b'+'{:06d}'.format(iorb)
    
    # BigDFT-tool does not like too long an argument, so we'll copy the input file to tmp.yaml
    shutil.copy(name+'.yaml', 'tmp.yaml')

    # now we can call BigDFT-tool
    cmd = '$BIGDFT_ROOT/bigdft-tool -a export-wf '+filename+' --name=tmp'+' > log-bigdft-tool.yaml'
    os.system(cmd)
    
    # for some reason the name from BigDFT and BigDFT-tool is not consistent, so we override it
    if tag == 'virtuals':
        tag = 'virtual'
    
    # we need to move the files into the data directory
    shutil.move(tag+'-k001-NR.b'+'{:06d}'.format(iorb)+'.cube', filename+'.cube')
    for xyz in ['x', 'y', 'z']:
        shutil.move(tag+'-k001-NR.b'+'{:06d}'.format(iorb)+'_avg_'+xyz+'.dat', filename+'_avg_'+xyz+'.dat')
    
    # finally we remove the temporary file we created
    os.remove('tmp.yaml')

Now we can use the function to convert to cube format, and calculate the spatial overlap. The value is high, indicating a local excitation.

In [10]:
import cubetools
        
data_dir = virt_run.data_directory
        
generate_cube(molecule, virt_run, name, ihomo)
cube_file = data_dir+'wavefunction-k001-NR.b'+'{:06d}'.format(ihomo)+'.cube'
datah, metah = cubetools.read_cube(cube_file)

# the virtual orbitals are indexed starting at 1
ilumo = 1
generate_cube(molecule, virt_run, name, ilumo, tag='virtuals')
cube_file = data_dir+'virtuals-k001-NR.b'+'{:06d}'.format(ilumo)+'.cube'
datal, metal = cubetools.read_cube(cube_file)

ct = np.sum(np.sqrt(datah**2 * datal**2))
print("Spatial overlap = "+'{0:.2f}'.format(ct))

Spatial overlap = 0.89


## STEP 4: Perform a T-CDFT Calculation with a Pure Constraint

We now have everything we need to perform a T-CDFT calculation, using the basis generated in step 1.

Despite still being farily pure, as noted, the $S_1$ excitation in naphthalene does display some contributions from transitions other than the HOMO-LUMO. As a first approximation we will first assume a pure HOMO-LUMO transition. We may then compare the results to the scenario where additional transitions are included.

First we need to define a new input file, starting from the one used to generate the basis.

In [11]:
from BigDFT import CDFT
from copy import deepcopy

# we'll make a copy of the input dictionary to be safe
inpc = deepcopy(inpv)

# the input follows along a fragment calculation, in that we do not want to further optimise the basis
inpc['lin_general'].update({'nit': [0, 1], 'extra_states': 0, 'rpnrm_cv': 1.0e-11, 'output_fragments': 2})
inpc['lin_basis'].update({'nit': 1, 'idsx': 0})
inpc['lin_kernel'].update({'nit': 100, 'rpnrm_cv': 1.0e-10, 'alphamix': 0.1, 'linear_method': 'DIAG'})
inpc.pop('output')
inpc.pop('mix')

Vc = -20.0

# now we can generate the constraints
s1 = CDFT.OpticalConstraint(kind='SINGLET')
s1.append_hole_component(fragment=1, orbital='HOMO', coefficient=1.0)
s1.append_electron_component(fragment=1, orbital='LUMO', coefficient=1.0)
s1.set_Vc(Vc)

t1 = CDFT.OpticalConstraint(kind='TRIPLET')
t1.append_hole_component(fragment=1, orbital='HOMO', coefficient=1.0)
t1.append_electron_component(fragment=1, orbital='LUMO', coefficient=1.0)
t1.set_Vc(Vc)

# as with any fragment calculation, we also need an extra directory layer
cdft_dir_name = 'tcdft'

Now let's set up the directories for the fragment approach.

In [12]:
import os

# we first create the directory
cdft_dir = 'data-'+cdft_dir_name
if not os.path.exists(cdft_dir):
    os.makedirs(cdft_dir)
                    
# arrange folders for fragment style calculation
basis_dir = molecule+'_'+str(xc)+'_S0_SF_'+basis_size+'_'+str(rloc)
os.chdir(cdft_dir)
if not os.path.exists('data-'+basis_dir):
    os.symlink('../data-'+basis_dir, 'data-'+basis_dir)
if not os.path.islink(basis_dir+'.xyz'): 
    os.symlink('../'+molecule+'.xyz', basis_dir+'.xyz')
os.chdir('../')

Our SF basis will be generated using a spin restricted calculation, while the T-CDFT triplet calculation will be performed in an unrestricted formalism. 

We therefore define a function which will use symbolic links to enable the T-CDFT calculation to read in the SFs for an unrestricted calculation, i.e. with both up and down SFs (which in this case are identical).

In [13]:
def link_support_functions(radical, nsfs):
    import os
        
    os.chdir('data-'+radical)
    
    for isf in range(nsfs):
        isfstr = str(isf + 1).zfill(6)
        # if we have the files already delete them to be safe
        if os.path.exists('minBasis-k001-UR.b'+isfstr):
            os.remove('minBasis-k001-UR.b'+isfstr)
        if os.path.exists('minBasis-k001-DR.b'+isfstr):
            os.remove('minBasis-k001-DR.b'+isfstr)
        # now make the links
        if not os.path.islink('minBasis-k001-UR.b'+isfstr):
            os.symlink('minBasis-k001-NR.b'+isfstr, 'minBasis-k001-UR.b'+isfstr)
        if not os.path.islink('minBasis-k001-DR.b'+isfstr):   
            os.symlink('minBasis-k001-NR.b'+isfstr, 'minBasis-k001-DR.b'+isfstr)
            
    os.chdir('../')

    
nsfs = lin_run.log['Total No. Support Functions']
link_support_functions(basis_dir, nsfs)

Let's now define a function to check the convergence. This is similar to the function used in the fragment tutorial, but in this case we also check whether the Lagrange multiplier was large enough to transfer the correct number of electrons.

In [14]:
def check_convergence(run, cdft=False, cdft_thresh=0.01):
    
    # in this case we check the convergence of the kernel loop
    dout = run.log['Ground State Optimization'][-1]['kernel optimization'][-1]['summary']['delta']

    # check what threshold we were trying to meet
    dthresh = float(run.log['lin_kernel']['rpnrm_cv'])
    
    converged = True
    
    if dout > dthresh:
        converged = False
        print('WARNING, calculation did not converge')
    
    if cdft:
        kw = run.log['Ground State Optimization'][-1]['kernel optimization'][-1]['summary']\
                    ['Constraint 1']['Tr(KW)']
        if abs(1.0 - kw) > cdft_thresh:
            converged = False
            print('WARNING, Tr(KW) != 1.0', kw)
            
    return converged

Finally we can run the singlet and triplet calculations.

In [15]:
import shutil

for state in ['S0', 'S1', 'T1']:
    
    # reset the CDFT part of the input
    if 'constrained_dft' in inpc:
        inpc.pop('constrained_dft', None)
                
    if state == 'S0':
        name = molecule+'_'+str(xc)+'_S0_SF_'+basis_size+'_'+str(rloc)
                    
        # don't run this calculation as a fragment calculation
        # otherwise the coefficients will not be written corectly
        inpc.update({'radical': basis_dir})
        inpc['dft'].update({'nspin': 1})
        inpc['lin_general'].update({'kernel_restart_mode': 'kernel', 'output_wf': 1, 'output_mat': 0})
        if 'frag' in inpc:
            inpc.pop('frag', None)
                
    else:
        # we'll include the constraint in the name, to avoid confusion with different constraints
        name = molecule+'_'+str(xc)+'_'+state+'_SF_'+basis_size+'_'+str(rloc)+'_HOMO_LUMO'
                    
        inpc.update({'radical': cdft_dir_name})
        inpc.update({'frag': {basis_dir: [1]}})
        inpc['lin_general'].update({'kernel_restart_mode': 'diag', 'output_wf': 0, 'output_mat': 1})

        if state == 'S1':
            inpc['dft'].update({'nspin': 1})
            inpc.add_cdft_constraint(s1, homo_per_fragment={1: ihomo})

        elif state == 'T1':
            inpc['dft'].update({'nspin': 2})
            inpc.add_cdft_constraint(t1, homo_per_fragment={1: ihomo})
            
    run = code.run(input=inpc, posinp=molecule+'.xyz', name=name) 

    if state == 'S0':
        s0_energy = Ha2eV * run.energy  
        converged = check_convergence(run)
    else: 
        energy = Ha2eV * run.energy - s0_energy
        converged = check_convergence(run, cdft=True)
        
        # in this case, we will reuse the kernel for the mixed constraint
        # so let's rename it to avoid overwriting it  
        src_file = 'data-'+cdft_dir_name+'/density_kernel_sparse.mtx'
        dest_file = 'data-'+cdft_dir_name+'/density_kernel_sparse_HOMO_LUMO_'+state+'.mtx'
        # we assume that if it exists already, it's the correct version
        # if it isn't the case, this step would need to be taken care of manually
        if not os.path.isfile(dest_file):
            shutil.move(src_file, dest_file)  
            
        # save the value for comparison with the mixed constraint
        if state == 'S1':
            pure_energy = energy

    time = run.log['Timings for root process']['Elapsed time (s)'] / 60.0  

    if state == 'S0':
        print(molecule+' S0 with '+xc+' and '+basis_size+' and Rloc = '+\
              str(rloc)+' took '+'{0:.1f}'.format(time)+' minutes, E = '+\
              '{0:.2f}'.format(s0_energy/run.nat)+' eV/atom')
    else:
        print(molecule+' '+state+' with '+xc+' and '+basis_size+' and Rloc = '+\
              str(rloc)+' took '+'{0:.1f}'.format(time)+' minutes, Delta E = '+\
              '{0:.2f}'.format(energy)+' eV')

naphthalene S0 with PBE and s_sp and Rloc = 8 took 0.3 minutes, E = -106.57 eV/atom
naphthalene S1 with PBE and s_sp and Rloc = 8 took 0.4 minutes, Delta E = 4.35 eV
naphthalene T1 with PBE and s_sp and Rloc = 8 took 1.0 minutes, Delta E = 2.97 eV


These values are slightly higher than the published values (see reference in the introduction), because we used a smaller basis set, but the difference is small.

## STEP 5: Perform a T-CDFT Calculation with a Mixed Constraint

Finally, we will repeat our T-CDFT calculation of $S_1$, including additional orbital contributions to $S_1$, guided by the TDDFT transition break down from Step 2.

First we run a T-CDFT calculation for each component of the targeted excitation. For the HOMO-LUMO component this will already have been run, but we will include it in the list of transitions for completeness.

In [16]:
# whether this is a singlet or triplet excitation
state = 'S1'

# here we define the orbitals involved in the transition, in this case coming from TDDFT
# we are assuming for now that we never have one hole -> more than one electron,
# although this can straightforwardly be generalised
electrons_holes = [['HOMO', 'LUMO'], ['HOMO-1', 'LUMO+1'], ['HOMO-2', 'LUMO+2']]
coefficients = [[1.0, 1.0], [1.0, 1.0], [1.0, 1.0]]
    
for eh,electron_hole in enumerate(electrons_holes):
    
    if 'constrained_dft' in inpc:
        inpc.pop('constrained_dft', None)

    if state[0] == 'S':
        constraint = CDFT.OpticalConstraint(kind='SINGLET')
        inpc['dft'].update({'nspin': 1})
    else:
        constraint = CDFT.OpticalConstraint(kind='TRIPLET')
        inpc['dft'].update({'nspin': 2})
      
    constraint.set_Vc(Vc)
    constraint.append_hole_component(fragment=1, orbital=electron_hole[0], coefficient=coefficients[eh][0])
    constraint.append_electron_component(fragment=1, orbital=electron_hole[1], coefficient=coefficients[eh][1])
    inpc.add_cdft_constraint(constraint, homo_per_fragment={1: ihomo})

    # run calculation
    name = molecule+'_'+str(xc)+'_SF_'+basis_size+'_'+str(rloc)+'_'+electron_hole[0]+'_'+electron_hole[1]
    run = code.run(input=inpc, posinp=molecule+'.xyz', name=name) 
    
    converged = check_convergence(run, cdft=True)
    
    energy = Ha2eV * run.energy - s0_energy
    
    # in this case, we will reuse the kernel for the mixed constraint
    # so let's rename it to avoid overwriting it  
    src_file = 'data-'+cdft_dir_name+'/density_kernel_sparse.mtx'
    dest_file = 'data-'+cdft_dir_name+'/density_kernel_sparse_'+electron_hole[0]+'_'+electron_hole[1]+\
                '_'+state+'.mtx'
    # we assume that if it exists already, it's the correct version
    # if it isn't the case, this step would need to be taken care of manually
    if not os.path.isfile(dest_file):
        shutil.move(src_file, dest_file)  

    # print energies, for information only
    print (molecule+' '+state+' '+electron_hole[0]+' -> '+electron_hole[1]+\
           ' took '+'{0:.1f}'.format(time)+' minutes, Delta E = '+'{0:.2f}'.format(energy)+' eV')

naphthalene S1 HOMO -> LUMO took 1.0 minutes, Delta E = 4.35 eV
naphthalene S1 HOMO-1 -> LUMO+1 took 1.0 minutes, Delta E = 6.16 eV
naphthalene S1 HOMO-2 -> LUMO+2 took 1.0 minutes, Delta E = 7.24 eV


In order to get the energy for the mixed constraint, we need to modify the input file.

In [17]:
# we'll copy and modify the input file for the pure constraints
inpm = deepcopy(inpc)

inpm['lin_general'].update({'kernel_restart_mode': 'full_kernel'})
# we want to run without any SCF iterations
# here for technical reasons we set nit = 1, otherwise the resulting logfile is not valid
inpm['lin_kernel'].update({'nit': 1})

# there is also no need any more to apply a constraint
inpm.pop('constrained_dft', None)

[{'Vc': -20.0,
  'constraint_type': 'OPTICAL',
  'hole': [{'fragment': 1, 'orbitals': {22: 1.0}}],
  'electron': [{'fragment': 1, 'orbitals': {27: 1.0}}],
  'excitation_type': 'SINGLET'}]

We also need to construct a new density kernel, weighting the constituent kernels based on the TDDFT transition breakdown.

In [18]:
from scipy.io import mmread, mmwrite

# define the transition weightings, based on the TDDFT breakdown
coefficients = [0.915, 0.028, 0.050]

# since this is not guaranteed to be normalised (due to missing small contributions), explicitly normalise it
coeff_sum = sum(coefficients)
coefficients = [c / coeff_sum for c in coefficients]
print('Running mixed T-CDFT with coefficients ', coefficients)

# combine the kernels
for eh,electron_hole in enumerate(electrons_holes):   
    kernel_file = 'data-'+cdft_dir_name+'/density_kernel_sparse'+'_'+\
                  electron_hole[0]+'_'+electron_hole[1]+'_S1.mtx'
    kernel = mmread(kernel_file)

    if eh == 0:
        combined_kernel = deepcopy(kernel)
        combined_kernel.data[:] = 0.0
        
    combined_kernel += coefficients[eh] * kernel     

mmwrite('data-'+cdft_dir_name+'/density_kernel_sparse.mtx', combined_kernel)

Running mixed T-CDFT with coefficients  [0.9214501510574018, 0.02819738167170191, 0.05035246727089627]


Finally, we can run the mixed calculation using this new kernel.

In [19]:
# for simplicity, we name the run simply with "mixed" as a suffix
# in the case where other trnasition breakdowns need to be tested, this would have to be modified
name = molecule+'_'+str(xc)+'_'+state+'_SF_'+basis_size+'_'+str(rloc)+'_mixed'
run = code.run(input=inpm, posinp=molecule+'.xyz', name=name) 

# take the energy from the first field where it's calculated
# instead of taking from run.energy, as this result is not correctly calculated in the case where nit=1
e = run.log['Ground State Optimization'][4]['support function optimization'][1]['Omega']['ENERGY']
mixed_energy = Ha2eV * e - s0_energy

print (molecule+' '+state+' with a mixed constraint took '+'{0:.1f}'.format(time)+\
       ' minutes, Delta E = '+'{0:.2f}'.format(mixed_energy)+' eV')
print (molecule+' '+state+' with a pure constraint Delta E = '+'{0:.2f}'.format(pure_energy)+' eV')

naphthalene S1 with a mixed constraint took 1.0 minutes, Delta E = 4.51 eV
naphthalene S1 with a pure constraint Delta E = 4.35 eV


As with the pure excitation, the energy is higher than the value given in the publication. This is both because of the smaller basis set, and because the transition breakdown was calculated using a smaller basis set.

In [ ]:
#Final remarks